In [10]:
import pandas as pd

In [11]:
def report_string_to_dict(report):
    report_dict = {}
    metrics = report.split('\n')[0].split()
    for row in report.split('\n')[1:]:
        splitted = row.split()
        if len(splitted) > 4:
            class_name_size = len(splitted[0:-4])
            class_name = " ".join(splitted[0:-4])
            report_dict[class_name] = {}
            for metric_index in range(len(metrics)):
                metric = metrics[metric_index]
                metric_val = splitted[metric_index + class_name_size]
                report_dict[class_name][metric] = metric_val
        elif len(splitted) > 0:
            metric = splitted[0]
            report_dict[metric] = {}
            report_dict[metric][metrics[-2]] = splitted[1]
            report_dict[metric][metrics[-1]] = splitted[2]
    return report_dict

In [12]:
def csv_to_dataframe(csv_name, csv_folder = ""):
    if len(csv_folder) == 0:
        return pd.read_csv(csv_name + ".csv")
    else:
        return pd.read_csv(csv_folder + "/" + csv_name + ".csv")